In [204]:
## Import packages
import numpy as np
import math
import random
from scipy.stats import poisson
import time

## Setup problem definition

## Test case

- T = 4h
- d = 5 min # the size of an interval
- beta = 9 minutes # service time (model by the exponential distribution)
- N = 24 patients
- I = T / d # the number of intervals

In [205]:
class schedule:
  def __init__(self, beta: int, T: int, d: int, N: int):
    self.beta = beta # = 1/μ : average service time
    self.T = T # number of intervals
    self.d = d # length of interval
    self.N = N # total number of patients
    self.x:list[int] = np.repeat(0, self.T) # schedule with x[t] as number of 
                                            # patients scheduled at the start of
                                            # interval t, t = 1,...,T; initially
                                            # set at zero
                                            
  def reset_schedule(self):
    for t in range(self.T):
      self.x[t] = 0

  def make_random_schedule(self, min_x: int, max_x: int, step: int=1):
    n = self.N
    for i in range(0,self.T, step):
      if n >= 0:
        r = random.randint(min_x, max_x)
        self.x[i] = min(r,n)
        n = n - r
        print(n, self.x[i])
      else:
        return
        
  def make_initial_schedule(self):
    for i in range(self.N):
      t = round(i*self.T / self.N)
      if i > self.T:
        i-= 1
      self.x[t] += 1

## Functions

In [206]:
# moving patients around iteratively 
# local search
def simpleSearch(x, p, beta, precision, precision_limit, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results, limit, v):
  for m in range(0, I-1):
    # for k in list(range(0, I))[::-1]: # [::-1]
    k = I-1
    while k >= 0:
      x_current = x.copy()
      if x_current[k] > 0: # Adding one vector is equivalent to moving the arrival of one patient from interval k to interval k+1
        next_k = (k + m + 1) % I

        x_current[k] -= 1
        x_current[next_k] += 1
        tic = time.perf_counter()
        temp_results = calcResults(x_current, p, beta, precision, precision_limit, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T, limit, v)
        toc = time.perf_counter()
        if temp_results['objVal'] < results['objVal']:
          x = x_current
          results = temp_results
          k += 1
          print(x)
          print(list(map(results.get, ['waitingTime', 'idleTime', 'tardiness'])), toc-tic)

        else: # undo the previous move
          x_current[k] += 1
          x_current[next_k] -= 1
          k -= 1
        
      else:
        k -= 1
  
  return x, results


In [207]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def create_p(beta, size, precision=0.9999): # Poisson distribution 
  k = 0
  p = []

  while sum(p) < precision: # fill accurate values up to precision limit
    p.append(poisson.pmf(k, beta))
    k+=1

  while len(p) < size: # fill the rest of the values with 0
    p.append(0)
  return p, k

In [208]:
def calcExponentialLimit(mu):
  return int(max(mu+4*mu**0.5,100))

In [209]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))

def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [210]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  for k in range(limit):
    tardiness += k * p_min[I][k] # I+1
  return tardiness


def calcIdletime(I, d, tardiness, n, no_show, beta):
  return (I * d) + tardiness - (n * (1 - no_show) * beta) # I-1?


def calcWaitingtime(p_min, x, p, limit, I, n):
  w = np.zeros((I+1, n+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k] 
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= n

  return waitingtime

In [211]:
def calculate_v(x, p, beta, precision, precision_limit, n, I, d, no_show=0):
    # count = len(p)
    count = precision_limit
    # waiting = 0
    # idletime = 0
    limit = calcExponentialLimit(beta*n) + 1
    p_plus = np.zeros((I+1, limit+d))
    p_min = np.zeros((I+1, limit+d))
    v = np.zeros((n+1, limit+d))
    add_v = np.zeros((n+1, limit+d))  # needs a predetermined size

    #	v[k][i]=probability of having i units of work given that
    #	k patients are scheduled for the interval.
    #	p[i]= probability of serving the patient in i mins given that
    #	the average service time is beta.

    add_v[0][0] = 1
    for k in range(1, n+1):
        limit = calcExponentialLimit(beta*k)
        i = 0
        sum_v = 0
        while sum_v < precision and i <= limit:
            z = 0
            while z <= count:
                add_v[k][i] += p[z] * add_v[k-1][i-z]
                z += 1
            sum_v += add_v[k][i] 
            i += 1

    for k in range(n+1):
        i = 0
        sum_v = 0
        while sum_v < precision and i <= limit:
            for m in range(k+1):
                v[k][i] += binomPMF(k, i, m, add_v, no_show)
            sum_v += v[k][i]
            i += 1
    return v, limit


precision = 0.9999
n = 24
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
s = schedule(beta, T, d, n)
s.make_random_schedule(1, 5)
x = s.x
# size of p has to be at least as big as the limit value here
size = calcExponentialLimit(beta*n)+1
p, precision_limit = create_p(beta, size, precision)
v, limit = calculate_v(x, p, beta, precision, precision_limit, n, I, d)


23 1
20 3
15 5
14 1
13 1
9 4
4 5
3 1
-1 3


In [212]:

def calculateProbabilities(x, p, beta, precision, precision_limit, limit, v, n, I, d):
  # count = len(p)
  count = precision_limit
  # waiting = 0
  # idletime = 0
  # limit = calcExponentialLimit(beta*n) + 1
  
  p_plus = np.zeros((I+1, calcExponentialLimit(beta*n)+1+d))
  p_min = np.zeros((I+1, calcExponentialLimit(beta*n)+1+d))
  #v = np.zeros((n+1,limit+d))
  #add_v = np.zeros((n+1,limit+d)) # needs a predetermined size

#	v[k][i]=probability of having i units of work given that 
#	k patients are scheduled for the interval.
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i] # when x[0] == 0 the probability calculations bug
    # probability of i minutes of work after interval 0 is equal to the probability of having i minutes of work given that 0 patients are scheduled
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1): # calculate p_min and p_plus iteratively 
    # Constraint 3
    for k in range(d+1):
      p_min[t][0] += p_plus[t-1][k]

    # Constraint 4
    for i in range(1,limit+1):
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

    # print(p_min[t])
    # print(p_plus[t])
  

  # Tardiness calcs
  # tardiness = calcTardiness(p_min, limit, I)

  # Idle time calcs new array of given shape and type, filled with zeros.
  # idletime = calcIdletime(I, d, tardiness, n, no_show, beta)

  # Waiting time calcs
  # waitingtime = calcWaitingtime(p_min, x, w, p, limit, I, n)

  # return p_min, waiting, idletime, tardiness
  return p_min, limit


### Test
precision = 0.9999
n = 24
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
s = schedule(beta, T, d, n)
s.make_random_schedule(1, 5)
x = s.x
# size of p has to be at least as big as the limit value here
size = calcExponentialLimit(beta*n)+1
p, precision_limit = create_p(beta, size, precision)
v, limit = calculate_v(x, p, beta, precision, precision_limit, n, I, d)
calculateProbabilities(x, p, beta, precision, precision_limit, limit, v, n, I, d)


23 1
21 2
16 5
14 2
9 5
4 5
2 2
-1 2


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.15690521e-01, 9.10903190e-02, 1.17116124e-01, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.76103105e-05, 1.15190132e-04, 3.18699073e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [8.36479902e-01, 1.58071555e-02, 1.47158494e-02, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [9.04804145e-01, 1.05881687e-02, 9.64819488e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [9.48822710e-01, 6.37735320e-03, 5.69048765e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 274)

In [213]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess

In [214]:
def calcResults(x, p, beta, precision, precision_limit, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T, limit, v):
  p_min, limit = calculateProbabilities(x, p, beta, precision, precision_limit, limit, v, n, I, d)
  # p_min, waitingTime, idleTime, tardiness = calculateProbabilities(x, p, beta, precision, precision_limit, n, no_show, I, d)
  
  # Tardiness calcs
  tardiness = calcTardiness(p_min, limit, I)

  # Idle time calcs new array of given shape and type, filled with zeros.
  idletime = calcIdletime(I, d, tardiness, n, no_show, beta)

  # Waiting time calcs
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, n)


  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  # Collect into a dictionary
  results = {'p_min' : p_min, 'waitingTime' : waitingtime, 'idleTime' : idletime, 'tardiness' : tardiness, 'objVal' : objVal}
  
  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess
  
  return results


# Main Function

In [218]:
precision = 0.9999
n = 24
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
# s = schedule(beta, T, d, n)
# s.make_random_schedule(1, 5)
# x = s.x
x = list(np.zeros(I, dtype=int)) # alternate x with all patients at the beginning
x[0] = n
# x = [0, 2, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
# size of p has to be at least as big as the limit value here
size = calcExponentialLimit(beta*n)+1
p, precision_limit = create_p(beta, size, precision)
v, limit = calculate_v(x, p, beta, precision, precision_limit, n, I, d)

no_show = 0
eind = 0

alpha_I = 0.2
alpha_T = 0.4 # patient doctor centric slider
alpha_W = 0.4


res = calcResults(x, p, beta, precision, precision_limit, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T, limit, v)

while res['objVal'] > 5:
    x, res = simpleSearch(x, p, beta, precision, precision_limit, n, no_show, I, d, alpha_W, alpha_I, alpha_T, res, limit,v)


[23, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[103.16846587730623, 24.332134627870204, 0.33213462787020853] 3.3554107990003104
[23, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[102.96053456632457, 24.33215574602289, 0.33215574602289205] 3.4612741139999343
[23, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[102.75260325534316, 24.332155941837925, 0.3321559418379287] 3.388262950999888
[23, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[102.5446719443616, 24.332155941837925, 0.3321559418379287] 3.42733525299991
[23, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

KeyboardInterrupt: 

Currently finishing in 55 mins, with the last 20 mins being no progress and just the final iterations.

# TODO evaluation metrics bugged when there is 1 or 0 in first interval? probably due to p_min problems

# TODO search can be finished but still has to perform I**2 number of probability calculations, introduce sane limit for objval?

# TODO optimal schedule is not found after one iteration of simpleSearch, efficiency of first patients moved depend on position of earlier patients in the list